Versione CUDA: 12.1
Versione pytorch: 2.3.0+cu121
Versione torchvision: 0.18.0+cu121

(pytorch e torchvision possono essere installate con il seguente comando da terminale:
"pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")

In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.3.0+cu121
0.18.0+cu121


In [2]:

import torch
 
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
 
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")
       
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? True
CUDA version: 12.1
ID of current CUDA device:0
Name of current CUDA device:NVIDIA GeForce MX230


Il seguente modello è quello di default di YOLO, e lo uso per fare il tracking dei giocatori:

Analizziamo il formato dell'output di model.predict, applicandolo ad un'immagine di prova:

In [4]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.predict('input_videos/img.png', save=True)

print(results)
print("OOOOOOOOOOOOOOOOO")
print(results[0])
print("boxes:")
for box in results[0].boxes:
    print(box)


image 1/1 d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\img.png: 480x640 9 persons, 1 dog, 30.0ms
Speed: 0.0ms preprocess, 30.0ms inference, 16.1ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict4
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: '

In [2]:
from ultralytics import YOLO
import cv2



# Apri il video di input
cap = cv2.VideoCapture('input.mp4')  # Sostituisci con il percorso del tuo video

# Ottieni le proprietà del video di input
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configura il writer per il video di output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Usa il codec mp4v per il formato MP4
out = cv2.VideoWriter('output_videos/output_video.mp4', fourcc, 24, (width, height))

# Esegui inferenza su un video con stream=True
model = YOLO('models/yolov8x.pt')

results = model.track("input_videos/input.mp4", stream=True, device='cuda')

# Processa i risultati e scrivi i frame nel video di output
for result in results:
    # Ottieni il frame originale
    frame = result.orig_img

    # Ottieni il frame con le annotazioni
    annotated_frame = result.plot()
    # La funzione plot() è una funzione fornita dall'oggetto risultato restituito dalla funzione predict() di YOLO.
    #La funzione plot() prende i risultati dell'inferenza (come i bounding box delle rilevazioni degli oggetti, 
    #le etichette degli oggetti, ecc.) e disegna queste annotazioni sul frame originale.

    # Scrivi il frame annotato nel video di output
    out.write(annotated_frame)

# Rilascia le risorse
cap.release()
out.release()

print("Il video annotato è stato salvato in output_video.mp4")


video 1/1 (frame 1/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 5 persons, 1 sports ball, 1 tennis racket, 2 clocks, 286.2ms
video 1/1 (frame 2/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 5 persons, 1 tennis racket, 2 clocks, 247.8ms
video 1/1 (frame 3/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 5 persons, 1 tennis racket, 2 clocks, 251.4ms
video 1/1 (frame 4/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 5 persons, 1 tennis racket, 2 clocks, 263.2ms
video 1/1 (frame 5/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 5 persons, 1 tennis racket, 2 clocks, 254.4ms
video 1/1 (frame 6/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 5 persons, 1 tennis racket, 2 clocks, 264.7ms
video 1/1 (frame 7/1994) d:\Li

Questo modello invece è quello da me addestrato per fare la detection della pallina da tennis:


Ho provato ad addestrare il modello usando varie versioni di yolo: ho provato sia la 8 sia la 5.
Della 8 ho provato due epochs diverse (20 e 100), mentre nella versione 5 ho provato solo con epochs = 100.
Ho poi estratto sia i pesi "best.pt" che quelli "last.pt" e provato ad applicarli su un video di prova per vedere quali dessero il risultato più preciso.
Alla fine i pesi contenenti in yolo5_last sono risultati i migliori e pertanto utilizzerò quelli.

In [1]:
from ultralytics import YOLO
import cv2

cap = cv2.VideoCapture('input_videos/input.mp4')  # Sostituisci con il percorso del tuo video

# Ottieni le proprietà del video di input
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configura il writer per il video di output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Usa il codec mp4v per il formato MP4
out = cv2.VideoWriter('output_videos/tennis_ball_detection_model_yolo8_best.mp4', fourcc, 24, (width, height))

model = YOLO('models/tennis_ball_detection_model_yolo8_best.pt')

results = model.predict("input_videos/input.mp4", conf=0.2, stream=True, device='cuda')

# Processa i risultati e scrivi i frame nel video di output
for result in results:
    # Ottieni il frame originale
    frame = result.orig_img

    # Ottieni il frame con le annotazioni
    annotated_frame = result.plot()
    # La funzione plot() è una funzione fornita dall'oggetto risultato restituito dalla funzione predict() di YOLO.
    #La funzione plot() prende i risultati dell'inferenza (come i bounding box delle rilevazioni degli oggetti, 
    #le etichette degli oggetti, ecc.) e disegna queste annotazioni sul frame originale.

    # Scrivi il frame annotato nel video di output
    out.write(annotated_frame)

# Rilascia le risorse
cap.release()
out.release()

print("Il video annotato è stato salvato nella cartella output_videos")



video 1/1 (frame 1/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 302.4ms
video 1/1 (frame 2/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 236.1ms
video 1/1 (frame 3/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 249.8ms
video 1/1 (frame 4/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 252.0ms
video 1/1 (frame 5/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 250.4ms
video 1/1 (frame 6/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 249.0ms
video 1/1 (frame 7/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 247.5ms
video 1/1 (frame 8/1994) d:\Linux

In [3]:
from ultralytics import YOLO
import cv2

cap = cv2.VideoCapture('input_videos/input.mp4')  # Sostituisci con il percorso del tuo video

# Ottieni le proprietà del video di input
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configura il writer per il video di output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Usa il codec mp4v per il formato MP4
out = cv2.VideoWriter('output_videos/old_tennis_ball_detection_model_yolo8_best.mp4', fourcc, 24, (width, height))

model = YOLO('models/old_tennis_ball_detection_model_yolo_8_best.pt')

results = model.predict("input_videos/input.mp4", conf=0.2, stream=True, device='cuda')

# Processa i risultati e scrivi i frame nel video di output
for result in results:
    # Ottieni il frame originale
    frame = result.orig_img

    # Ottieni il frame con le annotazioni
    annotated_frame = result.plot()
    # La funzione plot() è una funzione fornita dall'oggetto risultato restituito dalla funzione predict() di YOLO.
    #La funzione plot() prende i risultati dell'inferenza (come i bounding box delle rilevazioni degli oggetti, 
    #le etichette degli oggetti, ecc.) e disegna queste annotazioni sul frame originale.

    # Scrivi il frame annotato nel video di output
    out.write(annotated_frame)

# Rilascia le risorse
cap.release()
out.release()

print("Il video annotato è stato salvato nella cartella output_videos")



video 1/1 (frame 1/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 50.8ms
video 1/1 (frame 2/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 47.5ms
video 1/1 (frame 3/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 1 tennis ball, 53.0ms
video 1/1 (frame 4/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 1 tennis ball, 50.3ms
video 1/1 (frame 5/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 1 tennis ball, 38.1ms
video 1/1 (frame 6/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 45.6ms
video 1/1 (frame 7/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 1 tennis ball, 51.2ms
video 1/1 (frame 8/1994) d:\Linux\Universit\Anno

In [4]:

from ultralytics import YOLO
import cv2

cap = cv2.VideoCapture('input_videos/input.mp4')  # Sostituisci con il percorso del tuo video

# Ottieni le proprietà del video di input
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configura il writer per il video di output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Usa il codec mp4v per il formato MP4
out = cv2.VideoWriter('output_videos/tennis_ball_detection_model_yolo5_best.mp4', fourcc, 24, (width, height))

model = YOLO('models/tennis_ball_detection_model_yolo5_best.pt')

results = model.predict("input_videos/input.mp4", conf=0.2, stream=True, device='cuda')

# Processa i risultati e scrivi i frame nel video di output
for result in results:
    # Ottieni il frame originale
    frame = result.orig_img

    # Ottieni il frame con le annotazioni
    annotated_frame = result.plot()
    # La funzione plot() è una funzione fornita dall'oggetto risultato restituito dalla funzione predict() di YOLO.
    #La funzione plot() prende i risultati dell'inferenza (come i bounding box delle rilevazioni degli oggetti, 
    #le etichette degli oggetti, ecc.) e disegna queste annotazioni sul frame originale.

    # Scrivi il frame annotato nel video di output
    out.write(annotated_frame)

# Rilascia le risorse
cap.release()
out.release()

print("Il video annotato è stato salvato nella cartella output_videos")


c:\Users\turch\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


video 1/1 (frame 1/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 203.1ms
video 1/1 (frame 2/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 162.6ms
video 1/1 (frame 3/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 171.8ms
video 1/1 (frame 4/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 171.9ms
video 1/1 (frame 5/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 171.9ms
video 1/1 (frame 6/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 171.8ms
video 1/1 (frame 7/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 185.1ms
video 1/1 (frame 8/1994) d:\Linux\

In [5]:

from ultralytics import YOLO
import cv2

cap = cv2.VideoCapture('input_videos/input.mp4')  # Sostituisci con il percorso del tuo video

# Ottieni le proprietà del video di input
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configura il writer per il video di output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Usa il codec mp4v per il formato MP4
out = cv2.VideoWriter('output_videos/tennis_ball_detection_model_yolo5_last.mp4', fourcc, 24, (width, height))

model = YOLO('models/tennis_ball_detection_model_yolo5_last.pt')

results = model.predict("input_videos/input.mp4", conf=0.2, stream=True, device='cuda')

# Processa i risultati e scrivi i frame nel video di output
for result in results:
    # Ottieni il frame originale
    frame = result.orig_img

    # Ottieni il frame con le annotazioni
    annotated_frame = result.plot()
    # La funzione plot() è una funzione fornita dall'oggetto risultato restituito dalla funzione predict() di YOLO.
    #La funzione plot() prende i risultati dell'inferenza (come i bounding box delle rilevazioni degli oggetti, 
    #le etichette degli oggetti, ecc.) e disegna queste annotazioni sul frame originale.

    # Scrivi il frame annotato nel video di output
    out.write(annotated_frame)

# Rilascia le risorse
cap.release()
out.release()

print("Il video annotato è stato salvato nella cartella output_videos")



video 1/1 (frame 1/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 179.7ms
video 1/1 (frame 2/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 171.4ms
video 1/1 (frame 3/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 184.9ms
video 1/1 (frame 4/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 178.4ms
video 1/1 (frame 5/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 209.0ms
video 1/1 (frame 6/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 (no detections), 203.2ms
video 1/1 (frame 7/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input_videos\input.mp4: 384x640 1 tennis ball, 219.8ms
video 1/1 (frame 8/1994) d:\Linux\U